<a href="https://colab.research.google.com/github/CodeReber/big-data-challenge/blob/master/bigdatachallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-21 13:41:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.91MB/s    in 0.2s    

2020-11-21 13:41:44 (4.91 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

**Read Software tsv file from S3**

In [20]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://jim2020.s3.us-east-2.amazonaws.com/amazon_reviews_us_Software_v1_00.tsv"
spark.sparkContext.addFile(url)
amazon_software_review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv"), sep=r'\t', header=True, inferSchema=True)

# Show DataFrame
amazon_software_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [21]:
from pyspark.sql.functions import to_date
amz1_software_reviewid_df = amazon_software_review_df.select('customer_id', 'review_id', 'product_id', 'product_parent', to_date("review_date",'yyyy-MM-dd').alias("review_date")) 
amz1_software_reviewid_df.show()

+-----------+--------------+----------+--------------+-----------+
|customer_id|     review_id|product_id|product_parent|review_date|
+-----------+--------------+----------+--------------+-----------+
|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228| 2015-08-31|
|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352| 2015-08-31|
|   16053526|R1LSH74R9XAP59|B00164AZA4|     473982505| 2015-08-31|
|   15319481|R1QXUNTF76K7L6|B00E6LIEFM|     189774198| 2015-08-31|
|    1441820|R2F7DR75PS8NKT|B00VWEBG06|     852470365| 2015-08-31|
|   37107850|R2C1DJSCC8UFS6|B00EP7AP7C|     279360628| 2015-08-31|
|     302120|R1AXGS1W4YFXMX|B00OW2PET4|     729971168| 2015-08-31|
|   20193077|R1XU1B93402SYJ|B00N4OLCRO|     776572654| 2015-08-31|
|   13106017|R2U432NB3OPVR0|B005CELN8W|     222071424| 2015-08-31|
|   32587108|R3R6FIMIOQ5SP9|B005CELL1G|     168801430| 2015-08-31|
|   32020017|R12TX6V09C9QNQ|B00H09BOXQ|     455502995| 2015-08-31|
|   13376158|R33UCII6YKUMKV|B00MCLGAAO|     652069521| 2015-08

**Drop column not needed to prepare to be inserted in to RDS table**

In [ ]:
#prep for software tables insert
amz_software_reviewid_df = amazon_software_review_df.drop('marketplace','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body')
amz_software_reviewid_df.show()

+-----------+--------------+----------+--------------+-----------+
|customer_id|     review_id|product_id|product_parent|review_date|
+-----------+--------------+----------+--------------+-----------+
|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228| 2015-08-31|
|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352| 2015-08-31|
|   16053526|R1LSH74R9XAP59|B00164AZA4|     473982505| 2015-08-31|
|   15319481|R1QXUNTF76K7L6|B00E6LIEFM|     189774198| 2015-08-31|
|    1441820|R2F7DR75PS8NKT|B00VWEBG06|     852470365| 2015-08-31|
|   37107850|R2C1DJSCC8UFS6|B00EP7AP7C|     279360628| 2015-08-31|
|     302120|R1AXGS1W4YFXMX|B00OW2PET4|     729971168| 2015-08-31|
|   20193077|R1XU1B93402SYJ|B00N4OLCRO|     776572654| 2015-08-31|
|   13106017|R2U432NB3OPVR0|B005CELN8W|     222071424| 2015-08-31|
|   32587108|R3R6FIMIOQ5SP9|B005CELL1G|     168801430| 2015-08-31|
|   32020017|R12TX6V09C9QNQ|B00H09BOXQ|     455502995| 2015-08-31|
|   13376158|R33UCII6YKUMKV|B00MCLGAAO|     652069521| 2015-08

In [ ]:
amz_software_reviewid_df.describe()

DataFrame[summary: string, customer_id: string, review_id: string, product_id: string, product_parent: string, review_date: string]

In [ ]:
amz_software_reviewid_df.review_date

Column<b'review_date'>

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

In [ ]:
amz_software_reviewid_df.withColumn('review_date',col('review_date').cast(DateType()))

DataFrame[customer_id: int, review_id: string, product_id: string, product_parent: int, review_date: date]

In [16]:
#prep for software tables insert
amz_software_products_df = amazon_software_review_df.drop('marketplace','customer_id','review_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date')
new_df = amz_software_products_df.dropDuplicates(["product_id","product_title"])
new_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TQ71BXI|Moonlight Mysteri...|
|B00CHZG1FE|Image Line FL Stu...|
|B002JB3BC2|Anti-Virus & Anti...|
|B0113MZ1AC|Windows 8 64 Bit ...|
|B00003069T|ClueFinders 5th G...|
|B008EQUD4U|   Camtasia Studio 8|
|B008IVNFDM|Learn Adobe InDes...|
|B001Q24UAM|Play! The Ultimat...|
|B005HMUUSI|Mariner StoryMill...|
|B00005MG8G|My Deluxe MailLis...|
|B003L51CZ8|Sony Vegas Movie ...|
|B00JFYWY2Y|Learning SQL - Tr...|
|B00DDUDTZ2|Anno 2070 Complet...|
|B004625PV2|My Golf Game feat...|
|B0002TXT7K|Epic Pinball: The...|
|B001J8KF4I|Early Learning Fu...|
|B000KUWQ98|Extreme Behavior ...|
|B00O2QS9GU|Access 2013 Train...|
|B00NNU0C3Y|MAGIX Music Maker...|
|B00SLV70HQ|Can-Am Commander ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#prep for software tables insert
amz_software_customers_df = amazon_software_review_df.drop('marketplace','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date')
amz_software_customers_df.count()

341931

In [ ]:
#prep for software tables insert
amz_software_vine_df = amazon_software_review_df.drop('marketplace','customer_id','product_id','product_parent','product_title','product_category','verified_purchase','review_headline','review_body','review_date')
amz_software_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|
|R1XU1B93402SYJ|          1|            1|          1|   N|
|R2U432NB3OPVR0|          5|            0|          0|   N|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|
|R12TX6V09C9QNQ|          5|            0|          0|   N|
|R33UCII6YKUMKV|          3|            2|          2|   N|
| RZKDAB9TGO053|          1|            0|          0|   N|
|R2EMN2EEDN73ZA|          4|            

In [23]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatachallenge.c6c40hihvfwz.us-east-2.rds.amazonaws.com:5432/bigdatachallenge_db"
config = {"user":"root", 
          "password": "Welcome1", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to amz_software_vine_df in RDS

amz_software_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [18]:
# Write DataFrame to amz_software_products_df in RDS

new_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [24]:
# Write DataFrame to amz_software_reviewid_df in RDS

amz1_software_reviewid_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
amazon_software_review_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

**Read in Musical Instruments tsv file from S3**

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://jim2020.s3.us-east-2.amazonaws.com/amazon_reviews_us_Musical_Instruments_v1_00.tsv"
spark.sparkContext.addFile(url)
amazon_musical_review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv"), sep=r'\t', header=True, inferSchema=True)

# Show DataFrame
amazon_musical_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

**Prepare to be appended to table in RDS**

In [ ]:
#prep for software tables insert
amz_musical_reviewid_df = amazon_musical_review_df.drop('marketplace','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body')
amz_musical_reviewid_df.show()

+-----------+--------------+----------+--------------+-----------+
|customer_id|     review_id|product_id|product_parent|review_date|
+-----------+--------------+----------+--------------+-----------+
|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723| 2015-08-31|
|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292| 2015-08-31|
|    6111003| RIZR67JKUDBI0|B0006VMBHI|     603261968| 2015-08-31|
|    1546619|R27HL570VNL85F|B002B55TRG|     575084461| 2015-08-31|
|   12222213|R34EBU9QDWJ1GD|B00N1YPXW2|     165236328| 2015-08-31|
|   46018513|R1WCUI4Z1SIQEO|B001N4GRGS|     134151483| 2015-08-31|
|   10225065| RL5LNO26GAVJ1|B009PJRMHQ|     694166585| 2015-08-31|
|    6356995|R3GYQ5W8JHP8SB|B00NKBDAZS|     446431775| 2015-08-31|
|   35297198|R30SHYQXGG5EYC|B006MIU7U2|     125871705| 2015-08-31|
|   32139520|R14YLXA56NP51I|B000FIBD0I|     771888534| 2015-08-31|
|   36060782|R1ZH0HSH38IOTZ|B0002E52GG|      68535945| 2015-08-31|
|    5301309|R3H53KLLC210XI|B00RZIH52G|     725541773| 2015-08

In [ ]:
#prep for musical tables insert
amz_musical_products_df = amazon_musical_review_df.drop('marketplace','customer_id','review_id','product_parent','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date')
amz_musical_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HH62VB6|AGPtek® 10 Isolat...|
|B003LRN53I|Sennheiser HD203 ...|
|B0006VMBHI|AudioQuest LP rec...|
|B002B55TRG|Hohner Inc. 560BX...|
|B00N1YPXW2|Blue Yeti USB Mic...|
|B001N4GRGS|Middle Atlantic P...|
|B009PJRMHQ|Kmise 1pc Pickgua...|
|B00NKBDAZS|Kealoha Concert U...|
|B006MIU7U2|Halco 80000 - MR1...|
|B000FIBD0I|Gator GPTBLACK Pl...|
|B0002E52GG|Hetman 1 - Light ...|
|B00RZIH52G|Dragonpad pop fil...|
|B001792BAU|DharmaObjects Rel...|
|B009GSKW1Y|Musiclily SSS Pla...|
|B0002F4TKA|Vic Firth America...|
|B00K17YFBW|Guitar Stand for ...|
|B00EQ24HJS|Generic 3PLY Faux...|
|B00IBOYTUE|Audio 2000 6525 3...|
|B00FBRUSAE|Sawtooth ST-AMP-1...|
|B0113D2QUO|Upado Unlimited G...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#prep for musical tables insert
amz_musical_customers_df = amazon_musical_review_df.drop('marketplace','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date')
amz_musical_customers_df.count()

904765

In [ ]:
#prep for musical tables insert
amz_musical_vine_df = amazon_musical_review_df.drop('marketplace','customer_id','product_id','product_parent','product_title','product_category','verified_purchase','review_headline','review_body','review_date')
amz_musical_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|
| RZSL0BALIYUNU|          5|            0|          0|   N|
| RIZR67JKUDBI0|          3|            0|          1|   N|
|R27HL570VNL85F|          5|            0|          0|   N|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|
|R30SHYQXGG5EYC|          5|            0|          0|   N|
|R14YLXA56NP51I|          5|            1|          1|   N|
|R1ZH0HSH38IOTZ|          5|            0|          0|   N|
|R3H53KLLC210XI|          4|            0|          0|   N|
|R3OOR877NGA8JK|          3|            0|          0|   N|
|R1BY7WKOZ3KMH0|          2|            

Insert into RDS

In [ ]:
# Write DataFrame to amz_musical_vine_df in RDS

amz_musical_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
amazon_musical_review_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']